In [24]:
import numpy as np
import pandas as pd
import tensorflow as tf
from rdflib import Graph
from ampligraph.utils import save_model, restore_model
from ampligraph.evaluation import train_test_split_no_unseen
from ampligraph.latent_features import ScoringBasedEmbeddingModel
from ampligraph.latent_features.loss_functions import get as get_loss
from ampligraph.latent_features.regularizers import get as get_regularizer
from ampligraph.evaluation import mr_score, mrr_score, hits_at_n_score

In [6]:
tf.test.is_gpu_available()

2023-12-14 23:26:09.593676: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-14 23:26:09.593874: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-14 23:26:09.594267: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

True

In [7]:
g = Graph()
g.parse("../data/music_graph.ttl");

In [9]:
data =  [triplet for triplet in g]
len(data)

795348

In [11]:
cols = ["subject", "predicate", "object"]
triples_df = pd.DataFrame(data, columns=cols)
for c in cols:
    triples_df[c] = triples_df[c].apply(lambda x: x.toPython())
triples_df

,subject,predicate,object
0,http://www.semanticweb.org/music_ontology#trac...,http://www.semanticweb.org/music_ontology#name,Bad Dreams
1,http://www.semanticweb.org/music_ontology#trac...,http://www.semanticweb.org/music_ontology#danc...,0.181
2,http://www.semanticweb.org/music_ontology#even...,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://www.semanticweb.org/music_ontology#Event
3,http://www.semanticweb.org/music_ontology#arti...,http://www.semanticweb.org/music_ontology#gender,Male
4,http://www.semanticweb.org/music_ontology#trac...,http://www.semanticweb.org/music_ontology#partOf,http://www.semanticweb.org/music_ontology#albu...
...,...,...,...
795343,http://www.semanticweb.org/music_ontology#trac...,http://www.semanticweb.org/music_ontology#perf...,http://www.semanticweb.org/music_ontology#perf...
795344,http://www.semanticweb.org/music_ontology#trac...,http://www.semanticweb.org/music_ontology#spee...,0.0274
795345,http://www.semanticweb.org/music_ontology#even...,http://www.semanticweb.org/music_ontology#lati...,50.3701
795346,http://www.semanticweb.org/music_ontology#trac...,http://www.semanticweb.org/music_ontology#partOf,http://www.semanticweb.org/music_ontology#albu...


In [12]:
X_train, X_valid = train_test_split_no_unseen(np.array(data), test_size=10000)
print('Train set size: ', X_train.shape)
print('Test set size: ', X_valid.shape)

Train set size:  (785348, 3)
Test set size:  (10000, 3)


In [13]:
model = ScoringBasedEmbeddingModel(
    k=100, eta=1, scoring_type='TransE'
)

2023-12-14 23:29:45.664240: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-14 23:29:45.664844: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-14 23:29:45.664973: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [14]:
optim = tf.keras.optimizers.Adam(learning_rate=1e-4)
loss = get_loss('multiclass_nll')
regularizer = get_regularizer('LP', {'p': 3, 'lambda': 1e-5})

In [15]:
model.compile(
    optimizer=optim, loss=loss, entity_relation_regularizer=regularizer
)

In [16]:
# Fit the model
model.fit(
    X_train,
    batch_size=32768, # int(X_train.shape[0] / 50),
    epochs=1000,  # Number of training epochs
    verbose=True  # Enable stdout messages
);

Epoch 1/1000


2023-12-14 23:29:55.876677: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fcb983a41e0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-12-14 23:29:55.876695: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce GTX 1060 6GB, Compute Capability 6.1
2023-12-14 23:29:55.884289: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.


      2/Unknown - 3s 2s/step - loss: 22717.9688

2023-12-14 23:29:56.428719: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8700
2023-12-14 23:29:56.518764: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


25/25 [==============================] - 4s 156ms/step - loss: 22317.8594
Epoch 2/1000
 6/25 [======>.......................] - ETA: 0s - loss: 22216.6719

...

2023-12-14 23:43:52.127205: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 5640187878637657889


25/25 [==============================] - 1s 33ms/step - loss: 2128.6116
Epoch 1000/1000
 4/25 [===>..........................] - ETA: 0s - loss: 2128.4194

2023-12-14 23:43:52.978057: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 5640187878637657889


25/25 [==============================] - 1s 33ms/step - loss: 2127.0725


2023-12-14 23:43:53.835475: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 5640187878637657889


In [21]:
model_filename = "kg_embedding_model"
save_model(model, model_filename)

In [26]:
model = restore_model(model_filename)

Saved model does not include a db file. Skipping.


In [27]:
filter_triples = {'test': np.concatenate((X_train, X_valid))}
ranks = model.evaluate(
    X_valid, use_filter=filter_triples, verbose=True
)

314/314 [==============================] - 136s 434ms/step


2023-12-14 23:52:36.974258: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 7377645404474581255
2023-12-14 23:52:36.974385: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 11944932083158180741
2023-12-14 23:52:36.974393: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 11490521374090953855
2023-12-14 23:52:36.974398: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 15898939186316937977
2023-12-14 23:52:36.974403: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 215871133852060519
2023-12-14 23:52:36.974407: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 13866983229804850545
2023-12-14 23:52:36.974412: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv 

In [28]:
mr = mr_score(ranks)
mrr = mrr_score(ranks)
print(f"MRR: {mrr:.2f}")
print(f"MR: {mr:.2f}")

MRR: 0.22
MR: 5234.97


In [29]:
hits_10 = hits_at_n_score(ranks, n=10)
hits_3 = hits_at_n_score(ranks, n=3)
hits_1 = hits_at_n_score(ranks, n=1)
print(f"Hits@10: {hits_10:.2f}")
print(f"Hits@3: {hits_3:.2f}")
print(f"Hits@1: {hits_1:.2f}")

Hits@10: 0.28
Hits@3: 0.23
Hits@1: 0.19
